# Check Training Status

### Setup

In [1]:
import requests
import json
import sys


# To ease STDERR print
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)
    
    
# To ease checking api results since many return success HTTP code (200)
# but report true success/failure in a json object
def rspOk(rsp):
           
    if (rsp.ok):
        try:
            jsonBody = rsp.json()
            if ("result" in jsonBody) and ( jsonBody["result"] == "fail"):
                result = False
            else:
                result = True
        except ValueError:
            result = True
            eprint("WARN: good status_code, but no data")
    else:
        result = False
        
    return result

    
base_url = "http://9.5.95.52:9080/powerai-vision/api"


### Get list of tasks and show summary

In [2]:
from IPython.display import HTML, display, Image
import tabulate

url = base_url + "/dltasks/"
rsp = requests.get(url)
if (rspOk(rsp)):
    tasks = rsp.json()

# Adjust task list into summary table format
task_table = []
for i, task in enumerate(tasks):
    task_table.append((i, task["name"], task["status"], task["_id"], task["dataset_id"]))

display(HTML(tabulate.tabulate(task_table, headers=["Index", "Name", "Status", "_id", "dataset_id"], tablefmt='html')))

Index,Name,Status,_id,dataset_id
0,Dogs,deployed,6431e8c1-efe7-49a2-a2b8-a340c8aa7b75,04453c8d-a271-4ccd-aebf-7493417eca18
1,Car-Pedestrain-AutoLabeled,deployed,ee27eed0-9893-43c4-8f0e-f715a5f41ab0,06158d40-db07-4626-b833-419f96f27d76
2,Pedestrain-Car-Initial,deployed,e2d72c14-81ed-43b5-a332-efa656d62cbf,0f845ca5-c9f0-4a71-a04d-3c4cd62ecb45
3,Softdrinks,deployed,0403590e-f395-441c-9b69-b61bf6c6253c,80b1ba18-5b30-490a-b1b2-0a76f84de76b
4,Birds,deployed,c076da3f-a1f4-4686-8762-4d6896f25002,d2d95069-cbbb-4fbc-94ed-54d7e4b9dbab
5,Construction,deployed,6bfca0f8-aad6-4edd-8eb3-f28bf1a99a3c,666fee70-cd2f-47cc-a4ee-4c58a099170e


### Monitor status on specific training task
rerun cell to refresh status info

In [3]:
import time

monitor_index = 0
taskid_index = 3

url = base_url + "/dltasks/" + task_table[monitor_index][taskid_index] + "/status"
print(url)
rsp = requests.get(url)

if rspOk(rsp):
    status_list = rsp.json()
    print(json.dumps(status_list["data"][-1], indent=2))
else:
    eprint(rsp, rsp.text)

http://9.5.95.52:9080/powerai-vision/api/dltasks/6431e8c1-efe7-49a2-a2b8-a340c8aa7b75/status


<Response [200]> {"result":"fail"}


### show final task summary info

In [4]:
url = base_url + "/dltasks/" + task_table[monitor_index][taskid_index]
rsp = requests.get(url)
if (rspOk(rsp)):
    task = rsp.json()

# Adjust task list into summary table format
task_table = []
task_table.append((i, task["name"], task["status"], task["_id"], task["dataset_id"]))

display(HTML(tabulate.tabulate(task_table, headers=["Index", "Name", "Status", "_id", "dataset_id"], tablefmt='html')))

Index,Name,Status,_id,dataset_id
5,Dogs,deployed,6431e8c1-efe7-49a2-a2b8-a340c8aa7b75,04453c8d-a271-4ccd-aebf-7493417eca18


### Deploy model

In [5]:
url = base_url + "/webapis"
deploy_request = {"trained_model_id": task_table[monitor_index][taskid_index]}
rsp = requests.post(url, data=json.dumps(deploy_request))
if not rspOk(rsp):
    eprint("Request Failed")
    eprint(rsp)

### Test an Image

In [8]:
%ls -R 

.:
aivinference.ipynb  aiv.ipynb  aivsri.ipynb  dog1.jpg  dog2.jpg  dog3.jpg


In [10]:
filepath="dog3.jpg"
url = base_url + "/dlapis/" + task_table[monitor_index][taskid_index]
files = {'files': open(filepath, 'rb') }

rsp = requests.post(url, files=files)
if rspOk(rsp):
    result=rsp.json()
    print(json.dumps(result["classified"]))
    display(Image(url=result["imageUrl"], width=150, height=150))
    print(result["imageUrl"])
    print(json.dumps(result, indent=2))
else:
    eprint(rsp, rsp.text)

{"Weimaraner": "0.99752"}


http://9.5.95.52:9080/powerai-vision/uploads/temp/6431e8c1-efe7-49a2-a2b8-a340c8aa7b75/59ba0e8d-6aa6-4f14-b566-24f4f0b7a85d.jpg
{
  "imageUrl": "http://9.5.95.52:9080/powerai-vision/uploads/temp/6431e8c1-efe7-49a2-a2b8-a340c8aa7b75/59ba0e8d-6aa6-4f14-b566-24f4f0b7a85d.jpg",
  "imageMd5": "ca7e448df4e5de07627ad80734d7034d",
  "classified": {
    "Weimaraner": "0.99752"
  },
  "result": "success"
}
